In [1]:
#%%
import datetime
import pandas as pd
import numpy as np
import psycopg2
from utils.portfolio import *
from utils.base import *
from utils.report import *
from utils.config import *
from utils.instance import *
import os
import warnings
warnings.filterwarnings('ignore')



In [2]:
options, market = fetch_real_time_last_snapshot()
options = extract_call_options_features(options)
for i in range(len(options)):
    try:
      options['interest_rate_ask_price_1'][i]>1
    except:
        options['interest_rate_ask_price_1'][i] = np.nan

In [3]:
options.head(10)

,id_option,isin_option,symbol_option,name,open_price,close_price,last_trade_option,count,volume,value,...,min_allowed_price_base,in_the_money_percentage,ask_price_1_w_o_c,bid_price_1_w_o_c,leverage_ask_price_1,leverage_bid_price_1,leverage_last_trade,interest_rate_last_trade,interest_rate_ask_price_1,interest_rate_bid_price_1
0,31031620898020885,IRO9AGAH0151,ضترو4003,اختيارخ ص آگاه-9000-02/04/14,0.0,5213.0,5300.0,0,0.0,0.0,...,NaN,NaN,5520.0,4990.0,NaN,NaN,NaN,NaN,NaN,NaN
1,12833810923774304,IRO9AGAH0161,ضترو4004,اختيارخ ص آگاه-10000-02/04/14,4650.0,4650.0,4650.0,1,1.0,4650000.0,...,NaN,NaN,4650.0,4250.0,NaN,NaN,NaN,NaN,NaN,NaN
2,54559335299470096,IRO9AGAH0171,ضترو4005,اختيارخ ص آگاه-11000-02/04/14,0.0,3500.0,3500.0,0,0.0,0.0,...,NaN,NaN,4120.0,3300.0,NaN,NaN,NaN,NaN,NaN,NaN
3,54179059222245165,IRO9AGAH0181,ضترو4006,اختيارخ ص آگاه-12000-02/04/14,0.0,2824.0,2800.0,0,0.0,0.0,...,NaN,NaN,3280.0,2750.0,NaN,NaN,NaN,NaN,NaN,NaN
4,9181823518983931,IRO9AGAH0191,ضترو4007,اختيارخ ص آگاه-13000-02/04/14,2140.0,2232.0,2390.0,27,514.0,1147065000.0,...,NaN,NaN,2400.0,2300.0,NaN,NaN,NaN,NaN,NaN,NaN
5,63421907578271281,IRO9AGAH0201,ضترو4008,اختيارخ ص آگاه-14000-02/04/14,1440.0,1457.0,1700.0,8,51.0,74320000.0,...,NaN,NaN,1700.0,1422.0,NaN,NaN,NaN,NaN,NaN,NaN
6,6656725773350069,IRO9AGAH0221,ضترو4010,اختيارخ ص آگاه-16000-02/04/14,535.0,548.0,595.0,11,310.0,169900000.0,...,NaN,NaN,689.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN
7,59572243465235420,IRO9AGAH0231,ضترو4011,اختيارخ ص آگاه-18000-02/04/14,0.0,1.0,1.0,0,0.0,0.0,...,NaN,NaN,500.0,111.0,NaN,NaN,NaN,NaN,NaN,NaN
8,63758535312557533,IRO9AGAH0241,ضترو4012,اختيارخ ص آگاه-20000-02/04/14,0.0,1.0,1.0,0,0.0,0.0,...,NaN,NaN,400.0,99.0,NaN,NaN,NaN,NaN,NaN,NaN
9,29567585335146943,IRO9AHRM6291,ضهرم7003,اختيارخ اهرم-8000-1402/07/23,21000.0,21076.0,21500.0,4,59.0,1243501000.0,...,25480.0,0.715808,22000.0,21000.0,1.280865,1.339097,1.309302,0.505087,0.774686,0.290701


In [4]:
options.columns


Index(['id_option', 'isin_option', 'symbol_option', 'name', 'open_price',
       'close_price', 'last_trade_option', 'count', 'volume', 'value',
       'low_price', 'high_price', 'yesterday_price', 'table_id', 'group_id',
       'max_allowed_price', 'min_allowed_price_option', 'last_ret',
       'close_ret', 'number_of_shares', 'market_cap', 'bid_price_4',
       'bid_vol_4', 'ask_price_4', 'ask_vol_4', 'bid_price_2', 'bid_vol_2',
       'ask_price_2', 'ask_vol_2', 'bid_price_3', 'bid_vol_3', 'ask_price_3',
       'ask_vol_3', 'bid_price_5', 'bid_vol_5', 'ask_price_5', 'ask_vol_5',
       'bid_price_1', 'bid_vol_1', 'ask_price_1', 'ask_vol_1', 'ask_value_1',
       'ask_value_2', 'ask_value_3', 'ask_value_4', 'ask_value_5',
       'bid_value_1', 'bid_value_2', 'bid_value_3', 'bid_value_4',
       'bid_value_5', 'time', 'date', 'jalali_date', 'base_symbol',
       'jalali_due_date', 'days_until_due', 'miladi_due_date',
       'due_date_price', 'last_trade_base', 'symbol_base', 'id_base'

In [5]:
options['spread_pct'] = 100* (options['ask_price_1'] / options['bid_price_1']  - 1)

In [6]:
options[(options['value']>200e7 )& (options['spread_pct'] > 10) & \
        (options['last_trade_option'] /options['ask_price_1'] > 0.95 )&\
        (options['interest_rate_ask_price_1'] > 0.7)& (options['in_the_money_percentage'] > 0.2)]

,id_option,isin_option,symbol_option,name,open_price,close_price,last_trade_option,count,volume,value,...,in_the_money_percentage,ask_price_1_w_o_c,bid_price_1_w_o_c,leverage_ask_price_1,leverage_bid_price_1,leverage_last_trade,interest_rate_last_trade,interest_rate_ask_price_1,interest_rate_bid_price_1,spread_pct
100,17307087122635329,IRO9FOLD6921,ضفلا7030,اختيارخ فولاد-4969-1402/07/26,3005.0,2986.0,3600.0,16,639.0,2879715240.0,...,0.302107,3593.0,2930.0,1.983674,2.427534,1.977778,1.114927,1.101026,0.45068,22.375633


In [13]:
pd.options.display.max_rows = 10
options_selected = options[options['value']>500e7].sort_values('value', ascending=False)

In [15]:
options_selected['IV'] = 0
options_selected['call_bs'] = 0

In [40]:
for i in range(options_selected.shape[0]):
    options_selected['IV'].iloc[i] = implied_volatility_call(options_selected['last_trade_option'].iloc[i],
                   options_selected['last_trade_base'].iloc[i],
                   options_selected['due_date_price'].iloc[i],
                   options_selected['days_until_due'].iloc[i]/365,
                   0.3,)
    if options_selected['IV'].iloc[i] > 0 :
        options_selected['call_bs'].iloc[i] = black_scholes_call(options_selected['last_trade_base'].iloc[i],
                                                            options_selected['due_date_price'].iloc[i],
                                                            options_selected['days_until_due'].iloc[i]/365,
                                                            0.4, options_selected['IV'].iloc[i])
    else:
                options_selected['call_bs'].iloc[i] = black_scholes_call(options_selected['last_trade_base'].iloc[i],
                                                            options_selected['due_date_price'].iloc[i],
                                                            options_selected['days_until_due'].iloc[i]/365,
                                                            0.6, 0.4)


    print('done!')

found on 8th iteration
difference is equal to -1.7809472410590388e-05
done!
found on 10th iteration
difference is equal to -2.556117442509276e-05
done!
found on 19th iteration
difference is equal to -7.737053215350898e-05
done!
found on 30th iteration
difference is equal to -6.296118249338178e-05
done!
found on 4th iteration
difference is equal to -3.086947344854707e-06
done!
found on 31th iteration
difference is equal to 7.624302088515833e-05
done!
found on 25th iteration
difference is equal to -9.337197570857825e-05
done!
found on 27th iteration
difference is equal to -8.313550608818332e-05
done!
found on 14th iteration
difference is equal to -6.056489587535907e-05
done!
found on 11th iteration
difference is equal to -5.2032684379810235e-05
done!
found on 21th iteration
difference is equal to -9.257008332497207e-05
done!
done!
found on 6th iteration
difference is equal to -3.511580803206016e-05
done!
found on 7th iteration
difference is equal to -8.576747859478928e-06
done!
found on 

In [41]:
import plotly.express as px
options_selected = options_selected.reset_index(drop=True)
px.line(options_selected , x='symbol_option' , y='IV')

In [42]:
options_selected['diff'] = options_selected['last_trade_option'] - options_selected['call_bs']

In [43]:
px.line(options_selected , x='symbol_option' , y='diff')

In [47]:
pd.options.display.max_rows = 100

options_selected[options_selected['symbol_option']=='ضجار6000'].T

,27
id_option,52157458348940396
isin_option,IRO9BTEJ4551
symbol_option,ضجار6000
name,اختيارخ وتجارت-1500-1402/06/14
open_price,1333.0
close_price,1460.0
last_trade_option,1533.0
count,165
volume,7850.0
value,11457789000.0


In [70]:
black_scholes_call(2114,1500,135/365 , 0.75, 1.5)

1174.5177300541886

In [86]:
options_selected['ITM'] = options_selected['ITM'].astype(str)
px.scatter_3d(options_selected , x='days_until_due' , y='interest_rate_last_trade',z='in_the_money_percentage',
           hover_data=['symbol_option','diff'] ,color='ITM')

In [71]:
options_selected['ITM'] = 1

In [72]:
options_selected['ITM'][options_selected['due_date_price'] > options_selected['last_trade_option'] ] = 0

In [74]:
options_selected['interest_rate_last_trade'][options_selected['ITM']==1].mean()

0.737467581930548

In [81]:
options_selected['interest_rate_last_trade'][(options_selected['ITM']==0 )& (options_selected['interest_rate_last_trade']<np.inf)].mean()

2.3077022917246053

In [87]:
class BlackScholes:
    """ 
    Class to calculate (European) call and put option prices through the Black-Scholes formula 
    without dividends
    
    :param S: Price of underlying stock
    :param K: Strike price
    :param T: Time till expiration (in years)
    :param r: Risk-free interest rate (0.05 indicates 5%)
    :param sigma: Volatility (standard deviation) of stock (0.15 indicates 15%)
    """
    @staticmethod
    def _d1(S, K, T, r, sigma):
        return (1 / (sigma * np.sqrt(T))) * (np.log(S/K) + (r + sigma**2 / 2) * T)
    
    def _d2(self, S, K, T, r, sigma):
        return self._d1(S, K, T, r, sigma) - sigma * np.sqrt(T)
    
    def call_price(self, S, K, T, r, sigma):
        """ Main method for calculating price of a call option """
        d1 = self._d1(S, K, T, r, sigma)
        d2 = self._d2(S, K, T, r, sigma)
        return norm.cdf(d1) * S - norm.cdf(d2) * K * np.exp(-r*T)
    
    def put_price(self, S, K, T, r, sigma):
        """ Main method for calculating price of a put option """
        d1 = self._d1(S, K, T, r, sigma)
        d2 = self._d2(S, K, T, r, sigma)
        return norm.cdf(-d2) * K * np.exp(-r*T) - norm.cdf(-d1) * S
    
    def call_in_the_money(self, S, K, T, r, sigma):
        """ 
        Calculate probability that call option will be in the money at
        maturity according to Black-Scholes.
        """
        d2 = self._d2(S, K, T, r, sigma)
        return norm.cdf(d2)
    
    def put_in_the_money(self, S, K, T, r, sigma):
        """ 
        Calculate probability that put option will be in the money at
        maturity according to Black-Scholes.
        """
        d2 = self._d2(S, K, T, r, sigma)
        return 1 - norm.cdf(d2)

In [88]:
def call_implied_volatility(price, S, K, T, r):
    """ Calculate implied volatility of a call option up to 2 decimals of precision. """
    sigma = 0.0001
    while sigma < 1:
        d1 = BlackScholes()._d1(S, K, T, r, sigma)
        d2 = BlackScholes()._d2(S, K, T, r, sigma)
        price_implied = S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)
        if price - price_implied < 0.0001:
            return sigma
        sigma += 0.0001
    return "Not Found"

def put_implied_volatility(price, S, K, T, r):
    """ Calculate implied volatility of a put option up to 2 decimals of precision. """
    sigma = 0.0001
    while sigma < 1:
        call = BlackScholes().call_price(S, K, T, r, sigma)
        price_implied = K * np.exp(-r*T) - S + call
        if price - price_implied < 0.0001:
            return sigma
        sigma += 0.0001
    return "Not Found"

In [89]:
class OptionStrategies:
    @staticmethod
    def short_straddle(S, K, T, r, sigma):
        call = BlackScholes().call_price(S, K, T, r, sigma)
        put = BlackScholes().put_price(S, K, T, r, sigma)
        return - put - call
    
    @staticmethod
    def long_straddle(S, K, T, r, sigma):
        call = BlackScholes().call_price(S, K, T, r, sigma)
        put = BlackScholes().put_price(S, K, T, r, sigma)
        return put + call
    
    @staticmethod
    def short_strangle(S, K1, K2, T, r, sigma):
        assert K1 < K2, f"Please make sure that K1 < K2. Now K1={K1}, K2={K2}"
        put = BlackScholes().put_price(S, K1, T, r, sigma)
        call = BlackScholes().call_price(S, K2, T, r, sigma)
        return - put - call
    
    @staticmethod
    def long_strangle(S, K1, K2, T, r, sigma):
        assert K1 < K2, f"Please make sure that K1 < K2. Now K1={K1}, K2={K2}"
        put = BlackScholes().put_price(S, K1, T, r, sigma)
        call = BlackScholes().call_price(S, K2, T, r, sigma)
        return put + call
    
    @staticmethod
    def short_put_butterfly(S, K1, K2, K3, T, r, sigma):
        assert K1 < K2 < K3, f"Please make sure that K1 < K2 < K3. Now K1={K1}, K2={K2}, K3={K3}"
        put1 = BlackScholes().put_price(S, K1, T, r, sigma)
        put2 = BlackScholes().put_price(S, K2, T, r, sigma)
        put3 = BlackScholes().put_price(S, K3, T, r, sigma)
        return - put1 + 2 * put2 - put3
    
    @staticmethod
    def long_call_butterfly(S, K1, K2, K3, T, r, sigma):
        assert K1 < K2 < K3, f"Please make sure that K1 < K2 < K3. Now K1={K1}, K2={K2}, K3={K3}"
        call1 = BlackScholes().call_price(S, K1, T, r, sigma)
        call2 = BlackScholes().call_price(S, K2, T, r, sigma)
        call3 = BlackScholes().call_price(S, K3, T, r, sigma)
        return call1 - 2 * call2 + call3
    
    @staticmethod
    def short_iron_condor(S, K1, K2, K3, K4, T, r, sigma):
        assert K1 < K2 < K3 < K4, f"Please make sure that K1 < K2 < K3 < K4. Now K1={K1}, K2={K2}, K3={K3}, K4={K4}"
        put1 = BlackScholes().put_price(S, K1, T, r, sigma)
        put2 = BlackScholes().put_price(S, K2, T, r, sigma)
        call1 = BlackScholes().call_price(S, K3, T, r, sigma)
        call2 = BlackScholes().call_price(S, K4, T, r, sigma)
        return put1 - put2 - call1 + call2
    
    @staticmethod
    def long_iron_condor(S, K1, K2, K3, K4, T, r, sigma):
        assert K1 < K2 < K3 < K4, f"Please make sure that K1 < K2 < K3 < K4. Now K1={K1}, K2={K2}, K3={K3}, K4={K4}"
        put1 = BlackScholes().put_price(S, K1, T, r, sigma)
        put2 = BlackScholes().put_price(S, K2, T, r, sigma)
        call1 = BlackScholes().call_price(S, K3, T, r, sigma)
        call2 = BlackScholes().call_price(S, K4, T, r, sigma)
        return - put1 + put2 + call1 - call2

In [90]:
pip install blackscholes

Note: you may need to restart the kernel to use updated packages.
